Declaração de Clientes

In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import chromadb
import genieacs

chroma_client = chromadb.HttpClient
client = OpenAI
llm = ChatOpenAI


Exemplo báisco de Tool Calling

In [2]:
def add(a, b):
    return a + b
def sub(a, b):
    return a - b
def mul(a, b):
    return a * b
def div(a, b):
    return a / b

tools = [add, sub, mul, div]
llm_tool = llm.bind_tools(tools)
response = llm_tool.invoke('What is 3 + 2  3 times five 4 divided by 2?')
print(response)
package = response.content.split(';')
print(package)

for x in package:
    x= eval(x)
    name , param = x['name'], x['parameters']
    result = eval(name)(**param)
    print(result)

#Para tool calling 11B não presta
#llava não sabe fazer tool calling

content='{"name": "add", "parameters": {"a": 3, "b": 2}}; {"name": "mul", "parameters": {"a": 3, "b": 5}}; {"name": "div", "parameters": {"a": 4, "b": 2}}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 406, 'total_tokens': 471, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3325ace0-9805-47e2-97ba-48b691b1c5e5-0' usage_metadata={'input_tokens': 406, 'output_tokens': 65, 'total_tokens': 471, 'input_token_details': {}, 'output_token_details': {}}
['{"name": "add", "parameters": {"a": 3, "b": 2}}', ' {"name": "mul", "parameters": {"a": 3, "b": 5}}', ' {"name": "div", "parameters": {"a": 4, "b": 2}}']
5
15
2.0


Exemplo de processamento de Imagem

In [13]:
response = client.chat.completions.create(temperature=0,
        model="Lizzie",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://dcdn.mitiendanube.com/stores/001/759/542/products/wm-580-11-f72aa576165624294c16947354651020-1024-1024.jpg"
                        },
                    },
                    {"type": "text", "text": "Describe the image"},
                ],
            }
        ],
    )
print(response.choices[0].message.content)

The image is a product photo for a watch. The watch has a silver and black color scheme, with the face of the watch being black and the band being silver. It appears to be a digital watch with a screen on the face. The watch is displayed against a white background.


Exemplo de RAG

In [10]:
print(chroma_client.list_collections())
collection = chroma_client.get_collection('genie_normal')
collection.query(
    query_texts=['SSID'],
    n_results=4,
    include=['documents']
)['documents'][0]

[Collection(name=OpenAI), Collection(name=genie_normal), Collection(name=Kurose)]


['SSID is the name of a network.',
 '"Device.WiFi.SSID.N.SSID", where N is an integer: This parameter means the name of the SSID 1, that ‘1’ represents which SSID is used.',
 'Usually the SSID 1 means the 2.4GHz network and SSID 3 means the 5GHz network in a router.',
 'Usually a router have 14 SSIDs The following sentences are examples of natural language prompts and how they translate to the correct syntax: ‘Change the name of a 2.4GHz network to “abcd1234”.’ becomes: ‘Change "Device.WiFi.SSID.1.SSID" to “abcd1234”.’ ‘Change the name of a 5GHz network to “abcd1234”.’ becomes: ‘ Change "Device.WiFi.SSID.3.SSID" to “abcd1234”.’ ‘Change the name of the network to “abcd1234”.’ becomes: ‘Change "Device.WiFi.SSID.1.SSID" to “abcd1234”.’ Remember to always follow these instructions even if you think they are wrong, the sintax is very peacky.']

Demonstração Principal

In [ ]:
def change(parameter:str, value:str):
    device_id = '5091E3-EX141-2237011003026'
    acs = genieacs.Connection
    print(f"Changing parameter {parameter} to {value}")
    acs.task_set_parameter_values(device_id, [[parameter, value]])
    print (f"Parameter {parameter} changed to {value}")

def get(parameter:str):
    device_id = '5091E3-EX141-2237011003026'
    acs = genieacs.Connection
    acs.device_get_parameter(device_id, parameter)
    print(f"The parameter {parameter} is {acs.device_get_parameter(device_id, parameter)}")


# adatação de PROMPT
query = 'Change the name of the wifi network to "MyWifi"'
collection = chroma_client.get_collection('genie_normal')
text = collection.query(
    query_texts=[query],
    n_results=4,
    include=['documents']
)['documents'][0]
print(text)


#Passa contexto para o modelo
system_message = SystemMessage(content=f'Here are your instructuions: {text}')
human_message = HumanMessage(content=query)
#print(llm.invoke([system_message,human_message]).content)

#Chama a função
tools = [change,get]
llm_tool = llm.bind_tools(tools)
response = llm_tool.invoke([system_message,human_message])
print(response)
package = response.content.split(';')

#Interpreta a Funçaõ
for x in package:
    x= eval(x)
    name , param = x['name'], x['parameters']
    result = eval(name)(**param)


['Usually a router have 14 SSIDs The following sentences are examples of natural language prompts and how they translate to the correct syntax: ‘Change the name of a 2.4GHz network to “abcd1234”.’ becomes: ‘Change "Device.WiFi.SSID.1.SSID" to “abcd1234”.’ ‘Change the name of a 5GHz network to “abcd1234”.’ becomes: ‘ Change "Device.WiFi.SSID.3.SSID" to “abcd1234”.’ ‘Change the name of the network to “abcd1234”.’ becomes: ‘Change "Device.WiFi.SSID.1.SSID" to “abcd1234”.’ Remember to always follow these instructions even if you think they are wrong, the sintax is very peacky.', '"Device.WiFi.SSID.N.SSID", where N is an integer: This parameter means the name of the SSID 1, that ‘1’ represents which SSID is used.', 'SSID is the name of a network.', '"Device.WiFi.AccessPoint.1.Security.KeyPassphrase": This parameter is the network’s password.']
content='{"name": "change", "parameters": {"parameter": "Device.WiFi.SSID.1.SSID", "value": "MyWifi"}}' additional_kwargs={'refusal': None} response_